In [31]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *
import util_moments as yum

yu.setpath('analysis_B20')

enss_all=['b','c','d']

ffs_all=['A20','B20','C20']
js_conn=['j+;conn','j-;conn']

max_mom2_pc,max_mom2_pf=25,0 
n2qpp1s_conn=yum.get_n2qpp1s(max_mom2_pc,max_mom2_pf,noZeroQ=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
cases=['all','unequal','equal']
def run(n2qpp1,ff,j,enss):
    figs=[]
    for ens in enss:
        case2tf2ratio={case:{} for case in cases}
        path=f'/p/project1/ngff/li47/code/projectData/05_moments/doSVD/conn_{ens}_{yu.n2qpp12str(n2qpp1)}.h5'
        with h5py.File(path) as f:
            for case in f.keys():
                for key in f[case].keys():
                    tff,tj,tf=key.split('_'); tf=int(tf)
                    if (tff,tj) != (ff,j):
                        continue
                    case2tf2ratio[case][tf]=f[case][key][:]
                    
        cases_do=[case for case in cases if len(case2tf2ratio[case])!=0]
        
        
        def createDic(case):
            tf2ratio=case2tf2ratio[case]
            dic={
                'base:[tf2ratio,fits_band,fits_const,fits_sum,fits_2st]':[tf2ratio,None,None,None,None],
                'rainbow:[tfmin,tfmax,tcmin,dt]':[None,None,2,None],
            }
            return dic
        list_dic=[createDic(case) for case in cases_do]

        fig,axs=yu.makePlot_3ptasy(list_dic,shows=['rainbow','midpoint','fit_2st','fit_2st_prob','fit_sum','fit_band','fit_const'])
        fig.suptitle(rf'{yu.ens2label[ens]}; n2qpp1={n2qpp1}; $Q^2$={yum.n2qpp12Q2(n2qpp1,ens):.4f} GeV$^2$',fontsize=32)
        yu.addRowHeader(axs,cases_do)
        yu.finalizePlot(closeQ=True)
        figs.append(fig)

    yu.makePDF(f'{j}/{yu.n2qpp12str(n2qpp1)}_{ff}',figs,mkdirQ=True)

n2qpp1s=[(6,6,0),(12,12,0)]
ffs=ffs_all[1:2]
js=js_conn[:1]
enss=enss_all[:1]

for i1,n2qpp1 in enumerate(n2qpp1s):
    for i2,ff in enumerate(ffs):
        for i3,j in enumerate(js):
            print(f'{i1+1}/{len(n2qpp1s)} {n2qpp1}; {i2+1}/{len(ffs)} {ff}; {i3+1}/{len(js)} {j};',end='                   \r')
            run(n2qpp1,ff,j,enss)